In [1]:
!pip install solana solders aiohttp base58 jupyter pandas


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import asyncio
from solana.rpc.async_api import AsyncClient
from solders.pubkey import Pubkey
from solders.signature import Signature
import json
from datetime import datetime

# Test basic connection
async def test_connection():
    # Use free RPC endpoint (consider upgrading for production)
    client = AsyncClient("https://api.mainnet-beta.solana.com")
    
    try:
        # Get current slot
        slot_response = await client.get_slot()
        print(f"✅ Connected to Solana!")
        print(f"Current slot: {slot_response.value}")
        
        # Get recent blockhash
        blockhash = await client.get_latest_blockhash()
        print(f"Recent blockhash: {blockhash.value.blockhash}")
        
    except Exception as e:
        print(f"❌ Connection failed: {e}")
    finally:
        await client.close()

# Run this to test
await test_connection()

✅ Connected to Solana!
Current slot: 343283563
Recent blockhash: HWiRjqeZiWmDPwxo5NJk8TjwiosmoAnkRL8qKp8hR1MC


In [3]:
# Cell 3: Define important addresses
# Raydium AMM Program (where most memecoins create liquidity pools)
RAYDIUM_AMM_PROGRAM = "675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8"

# Raydium Authority
RAYDIUM_AUTHORITY = "5Q544fKrFoe6tsEbD7S8EmxGTJYAKtTVhAW5Q5pge4j1"

# Token Program
TOKEN_PROGRAM = "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA"

# SOL mint address (used to identify SOL in pairs)
SOL_MINT = "So11111111111111111111111111111111111111112"

print("Key addresses loaded ✅")

Key addresses loaded ✅


In [4]:
# Solana Memecoin Monitor - Step by Step Guide
# Run each cell one by one to understand how it works

# Cell 1: Install required packages (run in terminal or notebook)
"""
!pip install solana solders aiohttp base58 jupyter pandas
"""

# Cell 2: Import libraries and test connection
import asyncio
from solana.rpc.async_api import AsyncClient
from solders.pubkey import Pubkey
from solders.signature import Signature
import json
from datetime import datetime

# Test basic connection
async def test_connection():
    # Use free RPC endpoint (consider upgrading for production)
    client = AsyncClient("https://api.mainnet-beta.solana.com")
    
    try:
        # Get current slot
        slot_response = await client.get_slot()
        print(f"✅ Connected to Solana!")
        print(f"Current slot: {slot_response.value}")
        
        # Get recent blockhash
        blockhash = await client.get_latest_blockhash()
        print(f"Recent blockhash: {blockhash.value.blockhash}")
        
    except Exception as e:
        print(f"❌ Connection failed: {e}")
    finally:
        await client.close()

# Run this to test
await test_connection()

# Cell 3: Define important addresses
# Raydium AMM Program (where most memecoins create liquidity pools)
RAYDIUM_AMM_PROGRAM = "675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8"

# Raydium Authority
RAYDIUM_AUTHORITY = "5Q544fKrFoe6tsEbD7S8EmxGTJYAKtTVhAW5Q5pge4j1"

# Token Program
TOKEN_PROGRAM = "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA"

# SOL mint address (used to identify SOL in pairs)
SOL_MINT = "So11111111111111111111111111111111111111112"

print("Key addresses loaded ✅")

# Cell 4: Fetch recent transactions from Raydium
async def get_raydium_transactions(limit=10):
    client = AsyncClient("https://api.mainnet-beta.solana.com")
    
    try:
        # Get signatures for Raydium AMM program
        raydium_pubkey = Pubkey.from_string(RAYDIUM_AMM_PROGRAM)
        
        # Get recent signatures - using the correct method signature
        signatures = await client.get_signatures_for_address(
            raydium_pubkey,
            limit=limit
        )
        
        if hasattr(signatures, 'value') and signatures.value:
            print(f"Found {len(signatures.value)} recent Raydium transactions:")
            for i, sig_info in enumerate(signatures.value[:5]):  # Show first 5
                print(f"{i+1}. {sig_info.signature}")
            return signatures.value
        else:
            print("No signatures found")
            return []
            
    except Exception as e:
        print(f"Error fetching signatures: {e}")
        print(f"Error type: {type(e)}")
        return []
    finally:
        await client.close()

# Test fetching transactions
sigs = await get_raydium_transactions(limit=20)


✅ Connected to Solana!
Current slot: 343283636
Recent blockhash: GoHD3XAuyH4RTbDvSdLGTofhc8fZxbf8WCqtv84gcKFi
Key addresses loaded ✅
Found 20 recent Raydium transactions:
1. 4vWXGLSsp1vtsjhR8FMW7TDGuG8fDUcAP6UZpXnfXGYsNf3Lx7qP4LZaeeq8BPwDkM8kNw3Dq8snUZZmnmqbEuqs
2. 3k5Ma2qeaXLtrC8htuvrAJhhy1QGuqMP5fK7zgi2heCoZpd1W9saDcA7fg29Vv1kgU3heFEsSaJSHH3ZiBzoAcn2
3. 3vtBjWE2V4TkDMBRMdY2zYtragbPvGCTbYuDCPeK2RKhcYVtJhnZVynkvaRsPc1n9oc4FSzvRHBpMeCpp1VaDY8a
4. 22MmJtw8yZhM3QrHySNgMZkCJEepySBDwuoWnkdJQbNMV5vydT9hCPvVYqRyb49uxgfAQFW55rpNrtVyBRdh67Qt
5. 2njfYnZnYtx5bgCn5NvJXsuGWfyCKUwJkUHfdZ5F7TAxhkdsbtuSvDQUsrbqQNvRx4LHgAWNeTXbix34UUzcdogA


In [5]:
async def analyze_transaction(signature_str):
    client = AsyncClient("https://api.mainnet-beta.solana.com")
    
    try:
        # Convert string to Signature object
        sig = Signature.from_string(signature_str)
        
        # Get transaction details
        tx = await client.get_transaction(
            sig,
            encoding="jsonParsed",
            max_supported_transaction_version=0
        )
        
        if not tx or not hasattr(tx, 'value') or not tx.value:
            print("Transaction not found")
            return None
            
        transaction = tx.value
        
        # Extract basic info
        print(f"\n📋 Transaction: {signature_str[:20]}...")
        print(f"Slot: {transaction.slot}")
        print(f"Success: {transaction.meta.err is None}")
        
        # Look for instructions
        instructions = transaction.transaction.message.instructions
        print(f"Number of instructions: {len(instructions)}")
        
        # Check each instruction
        for i, inst in enumerate(instructions):
            program_id = str(inst.program_id)
            print(f"\nInstruction {i+1}:")
            print(f"  Program: {program_id[:20]}...")
            
            # Check if it's a Raydium instruction
            if program_id == RAYDIUM_AMM_PROGRAM:
                print("  ✅ This is a Raydium instruction!")
                
                # Try to get instruction type from data
                if hasattr(inst, 'data') and inst.data:
                    import base58
                    try:
                        decoded = base58.b58decode(inst.data)
                        # First 8 bytes are usually the instruction discriminator
                        discriminator = decoded[:8].hex()
                        print(f"  Discriminator: {discriminator}")
                    except:
                        print("  Could not decode instruction data")
        
        return transaction
        
    except Exception as e:
        print(f"Error analyzing transaction: {e}")
        return None
    finally:
        await client.close()

# Analyze the first transaction
if sigs:
    first_sig = str(sigs[0].signature)
    tx_data = await analyze_transaction(first_sig)


📋 Transaction: 4vWXGLSsp1vtsjhR8FMW...
Slot: 343283636
Error analyzing transaction: 'solders.transaction_status.EncodedConfirmedTransac' object has no attribute 'meta'
